In [2]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("../")

import os
from copy import copy

import numpy as np
import pandas as pd

from utils.generic_utils import pickle_in,pickle_out
import utils.preprocessing_utils as prep
from utils.evaluation_utils import rmsle,log_pandas,inv_log_pandas

In [3]:
kaggle_folder = os.path.join("../","datasets","kaggle_sets")
kaggle_test = pd.read_csv(os.path.join(kaggle_folder,"test.csv"))

initial_data_folder = os.path.join("../","datasets","initial_data_split")
initial_data_test = pd.read_csv(os.path.join(initial_data_folder,"test.csv"))

In [4]:
print kaggle_test.head()

              datetime  season  holiday  workingday  weather   temp   atemp  \
0  2011-01-20 00:00:00       1        0           1        1  10.66  11.365   
1  2011-01-20 01:00:00       1        0           1        1  10.66  13.635   
2  2011-01-20 02:00:00       1        0           1        1  10.66  13.635   
3  2011-01-20 03:00:00       1        0           1        1  10.66  12.880   
4  2011-01-20 04:00:00       1        0           1        1  10.66  12.880   

   humidity  windspeed  
0        56    26.0027  
1        56     0.0000  
2        56     0.0000  
3        56    11.0014  
4        56    11.0014  


In [6]:
X_test = kaggle_test
kaggle_datetime = kaggle_test[["datetime"]]

X_test_my = copy(initial_data_test).drop(["count","registered","casual"], axis=1)
Y_test_my = copy(initial_data_test)[["count"]]

full_pipeline = pickle_in(os.path.join("../","models","rf_simple_features_best_pipeline.pkl"),
                          compresion_mode=5)

Y_kaggle = pd.DataFrame(full_pipeline.predict(X_test)).apply(inv_log_pandas)
Y_test_my_pred = pd.DataFrame(full_pipeline.predict(X_test_my)).apply(inv_log_pandas)

test_error = rmsle(Y_test_my_pred.values.ravel(),
                   Y_test_my.values.ravel())
print "Score on my testset: %s\n"%test_error

kaggle_datetime["count"] = Y_kaggle

kaggle_datetime.to_csv(os.path.join("../","submissions",
                                    "rf_simple_features_best_submission.csv"),index=False)

print kaggle_datetime.head()

Score on my testset: 0.344339419963

             datetime      count
0 2011-01-20 00:00:00  10.027556
1 2011-01-20 01:00:00   3.805428
2 2011-01-20 02:00:00   2.564287
3 2011-01-20 03:00:00   3.069140
4 2011-01-20 04:00:00   2.553922


/home/jakubczakon/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
